In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D

import sys
sys.path.append('..')
from student_utils import adjacency_matrix_to_graph
from solver_utils import parse_input
from visualize import plot_graph
import collections

%matplotlib inline

In [ ]:
solvers = {
    'MST+DFS': mst_dfs_solve,
}

In [ ]:
"""
Solve Interface
Input:
    list_of_locations: A list of locations such that node i of the graph corresponds to name at index i of the list
    list_of_homes: A list of homes
    starting_car_location: The name of the starting location for the car
    adjacency_matrix: The adjacency matrix from the input file
Output:
    A list of locations representing the car path
    A dictionary mapping drop-off location to a list of homes of TAs that got off at that particular location
    NOTE: both outputs should be in terms of indices not the names of the locations themselves
"""

## Solver 1: MST + DFS

Idea: similar to the 2x approximation algorithm for MetricTSP, but in this case, we may not be able to remove every duplicate vertex seen (after its first occurence), because the original graph is not necessarily fully connected. Thus, we can only skip vertices when it is possible (i.e. there is an edge to take the shortcut). In this tour, we will drop everyone off at their house. No TA needs to walk.

Graphs that this is bad for:
- Branching graphs (lots of driving back and forth, where walking would be more optimal

Graphs that this is good for:
- Very connected graphs

In [ ]:
def mst_dfs_solve(list_of_locations,
                  list_of_homes,
                  starting_car_location,
                  adjacency_matrix,
                  params=[]):
    G, _ = adjacency_matrix_to_graph(adjacency_matrix)
    
    mst = nx.minimum_spanning_tree(G)
    seen = set()
    traversal = []
    def dfs(u):
        if u not in seen:
            traversal.append(u)
            seen.add(u)
            for v in mst.neighbors(u):
                if v not in seen:
                    dfs(v)
                    traversal.append(u)
    dfs(list_of_locations.index(starting_car_location))
    
    dropoffs = {
        home: [home] for home in list_of_homes 
    }
    return traversal

In [ ]:
input_path = './test_inputs/random_6v_3h.in'
(num_locations,
 num_houses,
 location_names,
 house_names,
 source,
 adj) = parse_input(input_path)

G, _ = adjacency_matrix_to_graph(adj)
traversal, dropoffs = mst_dfs_solve(location_names, house_names, source, adj)
print(traversal)

In [ ]:
edges_to_draw = []
for i in range(len(traversal) - 1):
    u, v = traversal[i], traversal[i + 1]
    edges_to_draw.append((u, v))

plot_graph(input_path,
           layout_style=nx.kamada_kawai_layout,
           show_edge_weights=True,
           edges_to_draw=edges_to_draw,
           directed=True)

In [ ]:
input_path = './test_inputs/branching_10v_5h.in'

## Solver 2: Dijkstra's at each vertex

In [ ]:
def dijkstra_greedy_solve(list_of_locations,
                          list_of_homes,
                          starting_car_location,
                          adjacency_matrix,
                          verbose=False,
                          params=[]):
    G, _ = adjacency_matrix_to_graph(adjacency_matrix)
    remaining = set([list_of_locations.index(name) for name in list_of_homes])
    source = curr = list_of_locations.index(starting_car_location)
    
    path = []
    
    # Continue looping until no more TAs to drop off
    while remaining:
        if verbose:
            print(f"\n==== AT NODE #{curr} ====")
        path.append(curr)
        
        if curr in remaining:
            if verbose:
                print(f"DROPPED TA OFF AT {curr}")
            remaining.remove(curr)

        # Which direction should we move (if at all)?
        heuristics = {}
        for n in G.neighbors(curr):
            distances, paths = nx.single_source_dijkstra(G, n)
            heuristics[n] = 0
            for h in remaining:
#                 if n == h:
#                     heuristics[n] += 2.0 # TODO: What should the heuristic be if you're at a house?
#                                          # Maybe just define as 1 / dist + 1
#                 else:
#                     heuristics[n] += 1 / distances[h]
                heuristics[n] += (1 / (distances[h] + 1)) ** 2 # 1/(dist+1)^2 heavily incentives closer houses
            if verbose:
                print(f"DISTANCES FROM {n}", distances)
        
        best_neighbor = max(list(heuristics.keys()), key=heuristics.get)
        if verbose:
            print("HEURISTICS: ", heuristics)
            print("BEST NEIGHBOR: ", best_neighbor)
        curr = best_neighbor

    if curr != source:
        sp = nx.shortest_path(G, source=curr, target=source, weight='weight')
        path += sp

    return path

In [ ]:
# input_path = './test_inputs/random_6v_3h.in'
input_path = '../phase1/50.in'
(num_locations,
 num_houses,
 location_names,
 house_names,
 source,
 adj) = parse_input(input_path)
G, _ = adjacency_matrix_to_graph(adj)
path = dijkstra_greedy_solve(location_names, house_names, source, adj)

In [ ]:
def draw_path(G, path):
    edges_to_draw = []
    for i in range(len(path) - 1):
        u, v = path[i], path[i + 1]
        edges_to_draw.append((u, v))
    plot_graph(input_path,
               layout_style=nx.kamada_kawai_layout,
               show_edge_weights=True,
               edges_to_draw=edges_to_draw,
               directed=True)

print("PATH: ", path)
draw_path(G, path)

In [ ]:
plot_graph(input_path,
           layout_style=nx.kamada_kawai_layout,
           show_edge_weights=True,
#            edges_to_draw=edges_to_draw,
           directed=True)

## Solver 3: Local Search

Start with either a random initial solution or the solution generated by the above greedy algorithm (that still visits all houses).

Evaluate over all "neighbors" of the solution, where a neighbor is defined as a closed walk that has an alternate path between one of its vertices. (This closed walk does NOT need to go through all vertices, and it can also traverse the same edge multiple times, and it can visit a vertex multiple times.

Swap the current solution with the best of the neighbors, and continue until no neighbors are superior to the current solution.

In [ ]:
from student_utils import cost_of_solution
import time

def k_shortest_paths(G, source, target, k, weight=None):
    from itertools import islice
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

def local_search_solve(list_of_locations,
                       list_of_homes,
                       starting_car_location,
                       adjacency_matrix,
                       params=[]):

    # Generate initial solution (random or greedy algorithm)
    current_solution = mst_dfs_solve(list_of_locations,
                                             list_of_homes,
                                             starting_car_location,
                                             adjacency_matrix,
                                             params=params)
#     current_solution = dijkstra_greedy_solve(list_of_locations,
#                                              list_of_homes,
#                                              starting_car_location,
#                                              adjacency_matrix,
#                                              params=params)
    G, _ = adjacency_matrix_to_graph(adjacency_matrix)
    
    home_idxs = [list_of_locations.index(h) for h in list_of_homes]
    all_pairs_dists = nx.floyd_warshall(G, weight='weight')
    
    def get_neighbors(G, path):
        """
        Returns "1-change" neighbors, which are all paths that:
            Either include a vertex that is not visited in path
            Include a vertex visited in path
            Swap a vertex that is visited with a vertex not visited
        """
        visited = set(path)
        unvisited = set(range(len(G))) - visited

        occurences = collections.Counter(path)

        neighbors = []

        # Exclude a vertex
        for vertex_to_exclude in visited - {0}:
            new_path = path.copy()
    #         print(f'VERTEX TO EXCLUDE: {vertex_to_exclude}')

            for i in range(occurences[vertex_to_exclude]):
                idx = new_path.index(vertex_to_exclude)
                prev, nxt = new_path[idx - 1], new_path[idx + 1]
                if prev == nxt:  # Able to skip over visiting
                    new_path.pop(idx)
                    new_path.pop(idx)
                else:
                    two_shortest = k_shortest_paths(G,
                                                    source=prev,
                                                    target=nxt,
                                                    k=2, weight='weight')
                    for alt_path in two_shortest:
                        if alt_path != path[idx-1:idx+2] and vertex_to_exclude not in alt_path:
                            new_path.pop(idx)
                            new_path = new_path[:idx-1] + alt_path + new_path[idx+1:]
                            break
                if vertex_to_exclude not in new_path:
#                     print(f'REMOVING {vertex_to_exclude}: {new_path}')
                    neighbors.append(new_path)
        
        # Include a vertex
        for vertex_to_include in unvisited:
            new_path = path.copy()

            one_away = []
            for v in visited:
                if vertex_to_include in G.neighbors(v):
                    one_away.append(v)
            if not one_away:
                continue
    #         print(f'\nVERTEX TO INCLUDE: {vertex_to_include}')
            closest_neighbor = min(one_away, key=lambda n: G[vertex_to_include][n]['weight'])
            # Get the index of the closest neighbor, check if a path exists between 
            # the vertex being added and the next vertex in the path. If so, a better
            # detour exists than the trivial path to the new vertex and directly back.
            closest_idx = new_path.index(closest_neighbor)
            if G.has_edge(vertex_to_include, new_path[closest_idx + 1]):
                new_path.insert(closest_idx + 1, vertex_to_include)
            else:
                new_path.insert(closest_idx + 1, closest_neighbor)
                new_path.insert(closest_idx + 1, vertex_to_include)
#             print(f'ADDING {vertex_to_include}: {new_path}')
            neighbors.append(new_path)

        # Swap a vertex (maybe add this)

        return neighbors

    def assign_dropoffs(G, path, home_idxs):
        """
        Returns the dictionary of all dropoffs along this path.
        """
        locations_on_path = set(path)
        dropoffs = collections.defaultdict(list)
    #     print(locations_on_path)
        for h in home_idxs:
    #         print(f'DISTANCES FOR {h}', all_pairs_dists[h])
            closest_loc_on_path = min(locations_on_path, key=lambda loc: all_pairs_dists[h][loc])
            dropoffs[closest_loc_on_path].append(h)
        return dropoffs

    def evaluate(G, path, home_idxs, verbose=False):
        """
        Assigns optimal dropoff locations for each TA, given a path that the car
        is going to take, and calculate the total energy expended. This is the
        metric that neighbors are going to be evaluated on.
        """
        dropoffs = assign_dropoffs(G, path, home_idxs)
        cost, msg = cost_of_solution(G, path, dropoffs, shortest=all_pairs_dists)
        if verbose:
            print(msg)

        return cost
    
#     print(get_neighbors(G, current_solution))
#     print(assign_dropoffs(G, current_solution, home_idxs))
    
    i = 0
    current_cost = evaluate(G, current_solution, home_idxs)
    
#     epsilon = 0.1
    epsilon = 0
    epsilon_decay_factor = 0.95
    while True:  # TODO(justinvyu): Add a timeout
        start_iter = time.time()
        neighbors = get_neighbors(G, current_solution.copy())
        
        # Take a random action with epsilon probability, decaying over time
        if np.random.rand() < epsilon:
            current_solution = neighbors[np.random.randint(len(neighbors))]
            current_cost = evaluate(G, current_solution, home_idxs)
            print(f'=== Iteration #{i}, Epsilon = {epsilon} ==='
                  f'\n Picked a RANDOM neighbor, Cost = {current_cost}')
            epsilon *= epsilon_decay_factor
            
        print(f'\nSearching over {len(neighbors)} neighbors...')
        best_neighbor = min(neighbors, key=(
            lambda neighbor_sol: evaluate(G, neighbor_sol, home_idxs)))
        best_cost = evaluate(G, best_neighbor, home_idxs, verbose=True)
        if best_cost < current_cost:
            print(f'=== Iteration #{i}, Epsilon = {epsilon} === \n Improved Cost = {best_cost}')
            current_solution = best_neighbor
            current_cost = best_cost
        else:
            print(f'Local search terminated with solution (cost={current_cost}): {current_solution}')
            return current_solution, best_cost
        i += 1
        epsilon *= epsilon_decay_factor
        end_iter = time.time()
        print(f'Iteration took {end_iter - start_iter} s')

In [ ]:
# input_path = '../phase1/100.in'
input_path = './test_inputs/branching_20v_5h.in'
(num_locations,
 num_houses,
 location_names,
 house_names,
 source,
 adj) = parse_input(input_path)
G, _ = adjacency_matrix_to_graph(adj)
# path = dijkstra_greedy_solve(location_names, house_names, source, adj)
path, best_cost = local_search_solve(location_names, house_names, source, adj)

edges_to_draw = []
for i in range(len(path) - 1):
    u, v = path[i], path[i + 1]
    edges_to_draw.append((u, v))
print(edges_to_draw)
plot_graph(input_path,
           layout_style=nx.kamada_kawai_layout,
           show_edge_weights=True,
           edges_to_draw=edges_to_draw,
           directed=True)

In [ ]:
G = nx.complete_graph(4)
path = [0, 1, 2, 3, 0]

def get_neighbors_alternate_paths(G, path):
    exhausted_vertices = set()
    # Track seen paths
    neighbors = set()
    for i in range(1, len(path)):
        if path[i] in exhausted_vertices:
            continue
        exhausted_vertices.add(path[i])
        
        seen_vertices = set()
        for j in range(i + 1, len(path)):
            if path[j] in seen_vertices:
                continue
            seen_vertices.add(path[j])
            
            path_so_far = path[:i]
            path_after = path[j+1:]
            print(f'path so far... {path_so_far}')
            print(f'all paths from {path[i]} to {path[j]}')
            shortest_simple_paths = k_shortest_paths(G,
                                                     source=path[i],
                                                     target=path[j],
                                                     k=2,
                                                     weight='weight')
            for alt_path in shortest_simple_paths:
#             for alt_path in nx.all_simple_paths(G, source=path[i], target=path[j], cutoff=2*(j - i)):
                if alt_path != path[i:j+1]:  # Don't want to include original as a neighbor
                    neighbors.add(tuple(path_so_far + alt_path + path_after))
                    print(alt_path, path[i:j+1])
                    break

            print(f'path after... {path_after}')
    neighbors.add((path[0], ))
    return neighbors
    
nx.draw(G, with_labels=True)
plt.show()
neighbors = get_neighbors_alternate_paths(G, path)

### Local Search experiments below

In [ ]:
def get_neighbors(G, path):
    """
    Returns "1-change" neighbors, which are all paths that:
        Either include a vertex that is not visited in path
        Include a vertex visited in path
        Swap a vertex that is visited with a vertex not visited
    """
    visited = set(path)
    unvisited = set(range(len(G))) - visited
    print(visited, unvisited)
    print(path)
    
    occurences = collections.Counter(path)
    
    neighbors = []
    
    # Exclude a vertex
    for vertex_to_exclude in visited - {0}:
        new_path = path.copy()
#         print(f'VERTEX TO EXCLUDE: {vertex_to_exclude}')

        for i in range(occurences[vertex_to_exclude]):
            idx = new_path.index(vertex_to_exclude)
            prev, nxt = new_path[idx - 1], new_path[idx + 1]
            if prev == nxt:  # Able to skip over visiting
                new_path.pop(idx)
                new_path.pop(idx)
            else:
                two_shortest = k_shortest_paths(G,
                                                source=prev,
                                                target=nxt,
                                                k=2, weight='weight')
                for alt_path in two_shortest:
                    if alt_path != path[idx-1:idx+2] and vertex_to_exclude not in alt_path:
                        new_path.pop(idx)
                        new_path = new_path[:idx-1] + alt_path + new_path[idx+1:]
                        break
            if vertex_to_exclude not in new_path:
                print(f'REMOVING {vertex_to_exclude}: {new_path}')
                neighbors.append(new_path)
    
    # Include a vertex
    for vertex_to_include in unvisited:
        new_path = path.copy()
        
        one_away = []
        for v in visited:
            if vertex_to_include in G.neighbors(v):
                one_away.append(v)
        if not one_away:
            continue
#         print(f'\nVERTEX TO INCLUDE: {vertex_to_include}')
        closest_neighbor = min(one_away, key=lambda n: G[vertex_to_include][n]['weight'])
        # Get the index of the closest neighbor, check if a path exists between 
        # the vertex being added and the next vertex in the path. If so, a better
        # detour exists than the trivial path to the new vertex and directly back.
        closest_idx = new_path.index(closest_neighbor)
        if G.has_edge(vertex_to_include, new_path[closest_idx + 1]):
            new_path.insert(closest_idx + 1, vertex_to_include)
        else:
            new_path.insert(closest_idx + 1, closest_neighbor)
            new_path.insert(closest_idx + 1, vertex_to_include)
        print(f'ADDING {vertex_to_include}: {new_path}')
        neighbors.append(new_path)
        
    # Swap a vertex (maybe add this)
        
    return neighbors

def assign_dropoffs(G, path, homes_idxs):
    """
    Returns the dictionary of all dropoffs along this path.
    """
    all_pairs_dists = nx.floyd_warshall(G, weight='weight')
#     print(all_pairs_dists)
    locations_on_path = set(path)
    dropoffs = collections.defaultdict(list)
#     print(locations_on_path)
    for h in homes_idxs:
#         print(f'DISTANCES FOR {h}', all_pairs_dists[h])
        closest_loc_on_path = min(locations_on_path, key=lambda loc: all_pairs_dists[h][loc])
        dropoffs[closest_loc_on_path].append(h)
    return dropoffs

G = nx.Graph()
G.add_weighted_edges_from([(0, 1, 1), (1, 2, 1), (0, 3, 1), (3, 4, 1), (3, 5, 2), (0, 4, 2)])
# path = [0, 1, 2, 1, 0, 3, 4, 3, 0]
print(G[4][0]['weight'])
path = [0, 1, 0, 3, 0]
home_idxs = [2, 3, 4]

# G = nx.complete_graph(4)
# path = [0, 1, 3, 0]

pos = nx.kamada_kawai_layout(G)
nx.draw(G, pos=pos, with_labels=True)
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=labels)
plt.show()
# get_neighbors(G, path)
assign_dropoffs(G, path, home_idxs)

In [ ]:
neighbors

In [ ]:
len(path)

In [ ]:
G = nx.Graph()
G.add_weighted_edges_from([(0, 1, 1), (1, 2, 1), (0, 3, 1), (3, 4, 1)])
nx.draw(G, with_labels=True)
# path = [0, 1, 2, 1, 0, 3, 4, 3, 0]
# path = [0, 1, 2, 1, 0]
path = [0, 1, 0, 3, 0]
# path = [0, 1, 0]
get_neighbors(G, path)

In [ ]:
[1, 2, 3][0:0]